In [38]:
!date

Wed Apr 16 04:39:16 PM EDT 2025


In [39]:
pwd

'/projects/sccn/andromeda1/aglinska/BC-ABCD-denoise-2/Code'

In [40]:
import os
import numpy as np
import pandas as pd
import ants
from matplotlib import pyplot as plt
import pickle
from tqdm import tqdm

In [41]:
def load_pickle(fn):
    if os.path.exists(fn):
        with open(fn, 'rb') as file:
            loaded_dict = pickle.load(file)
    return loaded_dict

In [53]:
indir = '../Data/StudyForrest/fmriprep/'
subs = [s for s in os.listdir(indir) if all((s.startswith('sub'),not s.endswith('.html')))]
n = len(subs)
n

14

In [54]:
analysis_name = 'DeepCor-Forrest-face-v1'
#analysis_name = 'YuOrig-00-orig'
analysis_dir = os.path.join('../Data/StudyForrest/ensembles_last_CVAE/',analysis_name)

In [55]:
events_fn_temp = '../Data/StudyForrest/events/{sub}_ses-localizer_task-objectcategories_run-{r}_events.tsv'

In [46]:
def correlate_columns(arr1, arr2):
    """
    Computes the Pearson correlation between corresponding columns of two matrices.
    
    Parameters:
    arr1 (np.ndarray): First matrix of shape (370, 1000)
    arr2 (np.ndarray): Second matrix of shape (370, 1000)
    
    Returns:
    np.ndarray: 1D array of correlations for each column (size 1000)
    """
    # Ensure input arrays are numpy arrays
    arr1 = np.asarray(arr1)
    arr2 = np.asarray(arr2)
    
    # Subtract the mean of each column (normalize)
    arr1_centered = arr1 - np.mean(arr1, axis=0)
    arr2_centered = arr2 - np.mean(arr2, axis=0)
    
    # Compute the numerator (covariance)
    numerator = np.sum(arr1_centered * arr2_centered, axis=0)
    
    # Compute the denominator (product of standard deviations)
    denominator = np.sqrt(np.sum(arr1_centered**2, axis=0) * np.sum(arr2_centered**2, axis=0))
    
    # Compute the Pearson correlation for each column
    correlation = numerator / denominator
    
    return correlation

In [33]:
def average_signal_maps(s,r):
    ofn_med = os.path.join(analysis_dir,f'signal_S{s}_R{r}_med.nii.gz')
    ofn_avg = os.path.join(analysis_dir,f'signal_S{s}_R{r}_avg.nii.gz')

    if not all([os.path.exists(ofn_med),os.path.exists(ofn_avg)]):
        signal_maps_fns = [os.path.join(analysis_dir,f) for f in os.listdir(analysis_dir) if f.startswith(f'signal_S{s}_R{r}_rep_')]
        signal_maps_fns.sort()
        signal_maps = [ants.image_read(signal_maps_fn) for signal_maps_fn in signal_maps_fns]
        arr = np.array([signal_map.numpy() for signal_map in signal_maps])
        signal_maps[0].new_image_like(np.median(arr,axis=0)).to_filename(ofn_med)
        signal_maps[0].new_image_like(np.average(arr,axis=0)).to_filename(ofn_avg)

In [36]:
def get_regs(events_fn):
    from nilearn.glm.first_level import make_first_level_design_matrix

    events = pd.read_csv(events_fn,delimiter='\t')

    t_r = 2.0 
    #n_scans = epi.shape[-1]
    n_scans = 156
    frame_times = (np.arange(n_scans) * t_r)

    X1 = make_first_level_design_matrix(frame_times,events,drift_model="polynomial",drift_order=3,hrf_model="SPM") #

    #face_reg = X1['face'].values
    #place_reg = X1[['body', 'house', 'object', 'scene', 'scramble']].values.sum(axis=1)

    #face_reg = X1[['face','body']].values.sum(axis=1)
    face_reg = X1[['face']].values.sum(axis=1)
    place_reg = X1[['house','scene']].values.sum(axis=1)
    
    return face_reg,place_reg

In [37]:
def map_corr2reg(epi,roi,reg,ofn):
    epi_flat = epi.numpy().reshape(-1,epi.shape[-1])
    gm_idx = roi.flatten()==1
    std1_idx = epi_flat.std(axis=1)>1e-3
    use_idx=gm_idx*std1_idx
    voxel_array = epi_flat[use_idx,:]
    voxel_array = (voxel_array-voxel_array.mean(axis=1)[:,np.newaxis]) / voxel_array.std(axis=1)[:,np.newaxis]
    reg_arr = np.repeat(reg[:,np.newaxis], voxel_array.shape[0], axis=1)
    corr_vals = correlate_columns(voxel_array.transpose(),reg_arr)
    corr_arr = np.zeros(epi_flat.shape[0])
    corr_arr[use_idx]=corr_vals
    corr_nii = roi.new_image_like(corr_arr.reshape(roi.shape))
    corr_nii.to_filename(ofn)

In [12]:
for s in tqdm(np.random.permutation(np.arange(n))):
    for r in [1,2,3,4]:
        average_signal_maps(s,r)

100%|██████████| 14/14 [13:43<00:00, 58.85s/it]


In [56]:
gm_fn = '../Data/StudyForrest/fmriprep/mask_roi.nii'
roi = ants.image_read(gm_fn)

In [57]:
for s in tqdm(range(14)):
    for r in [1,2,3,4]:
        sub=subs[s]
        face_reg,place_reg = get_regs(events_fn_temp.format(sub=sub,r=r))
        epi = ants.image_read(os.path.join(analysis_dir,f'signal_S{s}_R{r}_med.nii.gz'))
        map_corr2reg(epi,roi,face_reg,os.path.join(analysis_dir,f'signal-corr2face-S{s}_R{r}_med.nii.gz'))
        map_corr2reg(epi,roi,place_reg,os.path.join(analysis_dir,f'signal-corr2place-S{s}_R{r}_med.nii.gz'))

100%|██████████| 14/14 [01:28<00:00,  6.33s/it]


In [58]:
for s in tqdm(range(14)):
    image_files = [ants.image_read(os.path.join(analysis_dir,f'signal-corr2face-S{s}_R{r}_med.nii.gz')) for r in [1,2,3,4]]
    image_avg = image_files[0].new_image_like(np.array([image_file.numpy() for image_file in image_files]).mean(axis=0))
    image_avg.to_filename(os.path.join(analysis_dir,f'signal-corr2face-S{s}_R_avg_med.nii.gz'))
    
for s in tqdm(range(14)):
    image_files = [ants.image_read(os.path.join(analysis_dir,f'signal-corr2place-S{s}_R{r}_med.nii.gz')) for r in [1,2,3,4]]
    image_avg = image_files[0].new_image_like(np.array([image_file.numpy() for image_file in image_files]).mean(axis=0))
    image_avg.to_filename(os.path.join(analysis_dir,f'signal-corr2place-S{s}_R_avg_med.nii.gz'))

100%|██████████| 14/14 [00:00<00:00, 29.63it/s]


In [59]:
image_files = [ants.image_read(os.path.join(analysis_dir,f'signal-corr2face-S{s}_R_avg_med.nii.gz')) for s in range(14)]
image_avg =np.array([image_file.numpy() for image_file in image_files]).mean(axis=0)
image_files[0].new_image_like(image_avg).to_filename(os.path.join(analysis_dir,f'signal-corr2face-grand_average_avg_med.nii.gz'))

image_files = [ants.image_read(os.path.join(analysis_dir,f'signal-corr2place-S{s}_R_avg_med.nii.gz')) for s in range(14)]
image_avg =np.array([image_file.numpy() for image_file in image_files]).mean(axis=0)
image_files[0].new_image_like(image_avg).to_filename(os.path.join(analysis_dir,f'signal-corr2place-grand_average_avg_med.nii.gz'))